In [3]:
# ══════════════════════════════════════════════════════════════════════════════
# EXPLAINABLE MULTI-TASK LEARNING FOR SUPPLY CHAIN - FIXED VERSION
# ══════════════════════════════════════════════════════════════════════════════

import warnings
warnings.filterwarnings('ignore')

# Install required packages
!pip install -q xgboost lightgbm catboost shap imbalanced-learn optuna torch torchvision

# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime
import gc
from tqdm.auto import tqdm
import pickle
import json

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    roc_auc_score, precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report, f1_score, accuracy_score
)
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Tree-based models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim

# Explainability
import shap

# Imbalanced learning
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Optimization
import optuna
from optuna.samplers import TPESampler

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Determine XGBoost tree method
TREE_METHOD = 'hist'  # XGBoost 'gpu_hist' deprecated, use 'hist' with device='cuda'
USE_GPU = torch.cuda.is_available()

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 1: DATA LOADING & INITIAL EXPLORATION
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📊 PHASE 1: DATA LOADING & EXPLORATION")
print("="*80 + "\n")

# Load data
df = pd.read_csv('DataCo_cleaned.csv')

print(f"✅ Dataset loaded successfully!")
print(f"📈 Shape: {df.shape}")
print(f"📝 Columns: {df.columns.tolist()}\n")

# Basic info
print("🔍 Data Types:")
print(df.dtypes)
print(f"\n📊 Missing Values:")
print(df.isnull().sum())

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 2: ADVANCED FEATURE ENGINEERING
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔧 PHASE 2: ADVANCED FEATURE ENGINEERING")
print("="*80 + "\n")

# Create a copy for feature engineering
df_feat = df.copy()

# ──────────────────────────────────────────────────────────────────────────────
# 2.1 TEMPORAL FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("⏰ Creating Temporal Features...")

df_feat['Quarter'] = (df_feat['Month'] - 1) // 3 + 1
df_feat['Is_Weekend'] = df_feat['Day of Week'].isin([6, 7]).astype(int)
df_feat['Is_Month_Start'] = (df_feat['Week of Year'] % 4 == 1).astype(int)
df_feat['Is_Month_End'] = (df_feat['Week of Year'] % 4 == 0).astype(int)
df_feat['Season'] = df_feat['Month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})

# Cyclical encoding
df_feat['Month_Sin'] = np.sin(2 * np.pi * df_feat['Month'] / 12)
df_feat['Month_Cos'] = np.cos(2 * np.pi * df_feat['Month'] / 12)
df_feat['Week_Sin'] = np.sin(2 * np.pi * df_feat['Week of Year'] / 52)
df_feat['Week_Cos'] = np.cos(2 * np.pi * df_feat['Week of Year'] / 52)
df_feat['Day_Sin'] = np.sin(2 * np.pi * df_feat['Day of Week'] / 7)
df_feat['Day_Cos'] = np.cos(2 * np.pi * df_feat['Day of Week'] / 7)

print(f"✅ Created 12 temporal features")

# ──────────────────────────────────────────────────────────────────────────────
# 2.2 BEHAVIORAL FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("👥 Creating Behavioral Features...")

# Discount intensity
df_feat['Discount_Level'] = pd.cut(
    df_feat['Order Item Discount Rate'],
    bins=[-0.01, 0.0, 0.05, 0.15, 0.3, 1.0],
    labels=['No_Discount', 'Low', 'Medium', 'High', 'Very_High']
)

df_feat['High_Discount_Flag'] = (df_feat['Order Item Discount Rate'] > 0.15).astype(int)
df_feat['Extreme_Discount_Flag'] = (df_feat['Order Item Discount Rate'] > 0.3).astype(int)

# Price per unit
df_feat['Price_Per_Unit'] = df_feat['Sales'] / df_feat['Order Item Quantity'].replace(0, 1)

# Value bands
df_feat['Order_Value_Band'] = pd.cut(
    df_feat['Sales'],
    bins=[-np.inf, 50, 200, 500, 1000, np.inf],
    labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High']
)

# Quantity bands
df_feat['Quantity_Band'] = pd.cut(
    df_feat['Order Item Quantity'],
    bins=[0, 1, 3, 5, 10, np.inf],
    labels=['Single', 'Small', 'Medium', 'Large', 'Bulk']
)

print(f"✅ Created behavioral features")

# ──────────────────────────────────────────────────────────────────────────────
# 2.3 GEOGRAPHICAL & LOGISTICS FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("🌍 Creating Geographical Features...")

# Cross-border flag
df_feat['Is_Cross_Border'] = (
    df_feat['Customer Country'] != df_feat['Order Country']
).astype(int)

# International shipping
df_feat['Is_International'] = df_feat['Customer Country'].apply(
    lambda x: 0 if x in ['EE. UU.', 'United States'] else 1
)

# Geographic distance proxy
df_feat['Customer_Order_Same_Region'] = (
    df_feat['Customer Country'] == df_feat['Order Country']
).astype(int)

# Shipping mode risk
shipping_risk_map = {
    'Standard Class': 1,
    'Second Class': 2,
    'First Class': 3,
    'Same Day': 4
}
df_feat['Shipping_Risk_Score'] = df_feat['Shipping Mode'].map(shipping_risk_map).fillna(1)

print(f"✅ Created geographical features")

# ──────────────────────────────────────────────────────────────────────────────
# 2.4 PAYMENT & RISK PROXY FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("💳 Creating Payment & Risk Features...")

# Payment type risk scoring
payment_risk_map = {
    'PAYMENT': 1,
    'DEBIT': 2,
    'TRANSFER': 3,
    'CASH': 4
}
df_feat['Payment_Risk_Score'] = df_feat['Type'].map(payment_risk_map)

# High-risk payment combinations
df_feat['Cash_High_Discount'] = (
    (df_feat['Type'] == 'CASH') & (df_feat['High_Discount_Flag'] == 1)
).astype(int)

df_feat['Cash_Cross_Border'] = (
    (df_feat['Type'] == 'CASH') & (df_feat['Is_Cross_Border'] == 1)
).astype(int)

df_feat['Rush_Shipping_High_Discount'] = (
    (df_feat['Shipping Mode'] == 'Same Day') & (df_feat['High_Discount_Flag'] == 1)
).astype(int)

# Composite fraud risk proxy
df_feat['Fraud_Risk_Proxy_Score'] = (
    df_feat['Payment_Risk_Score'] * 0.3 +
    df_feat['High_Discount_Flag'] * 2.0 +
    df_feat['Is_Cross_Border'] * 1.5 +
    df_feat['Shipping_Risk_Score'] * 0.5 +
    df_feat['Extreme_Discount_Flag'] * 3.0
)

# Binary fraud proxy
fraud_threshold = df_feat['Fraud_Risk_Proxy_Score'].quantile(0.75)
df_feat['Fraud_Risk_Label'] = (
    df_feat['Fraud_Risk_Proxy_Score'] > fraud_threshold
).astype(int)

print(f"✅ Fraud Risk Distribution:")
print(df_feat['Fraud_Risk_Label'].value_counts())
print(f"   Fraud Rate: {df_feat['Fraud_Risk_Label'].mean()*100:.2f}%")

# ──────────────────────────────────────────────────────────────────────────────
# 2.5 AGGREGATION FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("📊 Creating Aggregation Features...")

# Customer-level aggregations
customer_agg = df_feat.groupby('Customer City').agg({
    'Order Item Quantity': ['mean', 'std', 'sum'],
    'Sales': ['mean', 'std', 'sum'],
    'Order Item Discount Rate': 'mean',
    'Fraud_Risk_Label': 'mean'
}).reset_index()
customer_agg.columns = ['Customer City',
                        'Customer_Avg_Quantity', 'Customer_Std_Quantity', 'Customer_Total_Quantity',
                        'Customer_Avg_Sales', 'Customer_Std_Sales', 'Customer_Total_Sales',
                        'Customer_Avg_Discount', 'Customer_Fraud_Rate']

df_feat = df_feat.merge(customer_agg, on='Customer City', how='left')

# Product-level aggregations
product_agg = df_feat.groupby('Product Name').agg({
    'Order Item Quantity': ['mean', 'sum'],
    'Sales': ['mean', 'sum'],
    'Fraud_Risk_Label': 'mean'
}).reset_index()
product_agg.columns = ['Product Name',
                       'Product_Avg_Quantity', 'Product_Total_Quantity',
                       'Product_Avg_Sales', 'Product_Total_Sales',
                       'Product_Fraud_Rate']

df_feat = df_feat.merge(product_agg, on='Product Name', how='left')

# Region-level aggregations
region_agg = df_feat.groupby('Order Region').agg({
    'Order Item Quantity': ['mean', 'sum'],
    'Sales': ['mean', 'sum'],
    'Fraud_Risk_Label': 'mean'
}).reset_index()
region_agg.columns = ['Order Region',
                      'Region_Avg_Quantity', 'Region_Total_Quantity',
                      'Region_Avg_Sales', 'Region_Total_Sales',
                      'Region_Fraud_Rate']

df_feat = df_feat.merge(region_agg, on='Order Region', how='left')

# Category-level aggregations
category_agg = df_feat.groupby('Category Name').agg({
    'Order Item Quantity': ['mean', 'sum'],
    'Sales': ['mean', 'sum'],
    'Fraud_Risk_Label': 'mean'
}).reset_index()
category_agg.columns = ['Category Name',
                        'Category_Avg_Quantity', 'Category_Total_Quantity',
                        'Category_Avg_Sales', 'Category_Total_Sales',
                        'Category_Fraud_Rate']

df_feat = df_feat.merge(category_agg, on='Category Name', how='left')

print(f"✅ Created aggregation features")

# ──────────────────────────────────────────────────────────────────────────────
# 2.6 INTERACTION FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("🔗 Creating Interaction Features...")

df_feat['Discount_X_Quantity'] = df_feat['Order Item Discount Rate'] * df_feat['Order Item Quantity']
df_feat['Price_X_Discount'] = df_feat['Price_Per_Unit'] * df_feat['Order Item Discount Rate']
df_feat['Region_X_Payment_Risk'] = df_feat['Region_Fraud_Rate'] * df_feat['Payment_Risk_Score']

print(f"✅ Created interaction features")

# ──────────────────────────────────────────────────────────────────────────────
# 2.7 MONTHLY DEMAND AGGREGATION
# ──────────────────────────────────────────────────────────────────────────────
print("📅 Creating Monthly Demand Target...")

# Create Product × Region × Time key
df_feat['Product_Region_Key'] = (
    df_feat['Product Name'].astype(str) + '_' +
    df_feat['Order Region'].astype(str)
)

df_feat['Year_Month'] = df_feat['Year'].astype(str) + '_' + df_feat['Month'].astype(str).str.zfill(2)

# Monthly demand aggregation
monthly_demand = df_feat.groupby(['Product_Region_Key', 'Year_Month']).agg({
    'Order Item Quantity': 'sum',
    'Sales': 'sum'
}).reset_index()

monthly_demand.columns = ['Product_Region_Key', 'Year_Month', 'Monthly_Demand', 'Monthly_Sales']

# Merge back
df_feat = df_feat.merge(monthly_demand, on=['Product_Region_Key', 'Year_Month'], how='left')

print(f"✅ Monthly demand aggregated")
print(f"   Unique Product-Region combinations: {df_feat['Product_Region_Key'].nunique()}")
print(f"   Demand range: {df_feat['Monthly_Demand'].min():.0f} - {df_feat['Monthly_Demand'].max():.0f}")

print(f"\n✅ FEATURE ENGINEERING COMPLETE!")
print(f"📊 Final dataset shape: {df_feat.shape}")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 3: DATA PREPROCESSING & ENCODING
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔨 PHASE 3: DATA PREPROCESSING & ENCODING")
print("="*80 + "\n")

# Create a copy for modeling
df_model = df_feat.copy()

# ──────────────────────────────────────────────────────────────────────────────
# 3.1 HANDLE CATEGORICAL VARIABLES
# ──────────────────────────────────────────────────────────────────────────────
print("🏷️ Encoding Categorical Variables...")

# Define categorical columns
categorical_cols = [
    'Type', 'Category Name', 'Customer Country', 'Customer Segment',
    'Department Name', 'Market', 'Order Country', 'Order Region',
    'Shipping Mode', 'Season', 'Discount_Level', 'Order_Value_Band',
    'Quantity_Band'
]

# Label encoding
label_encoders = {}
for col in categorical_cols:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[f'{col}_Encoded'] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le

print(f"✅ Encoded {len(categorical_cols)} categorical features")

# ──────────────────────────────────────────────────────────────────────────────
# 3.2 SELECT FEATURES FOR MODELING
# ──────────────────────────────────────────────────────────────────────────────
print("🎯 Selecting Features for Modeling...")

# Numerical features
numerical_features = [
    'Days for shipment (scheduled)', 'Order Item Discount Rate', 'Order Item Quantity',
    'Sales', 'Day of Week', 'Month', 'Year', 'Week of Year', 'Quarter',
    'Is_Weekend', 'Is_Month_Start', 'Is_Month_End',
    'Month_Sin', 'Month_Cos', 'Week_Sin', 'Week_Cos', 'Day_Sin', 'Day_Cos',
    'High_Discount_Flag', 'Extreme_Discount_Flag', 'Price_Per_Unit',
    'Is_Cross_Border', 'Is_International', 'Customer_Order_Same_Region',
    'Shipping_Risk_Score', 'Payment_Risk_Score',
    'Cash_High_Discount', 'Cash_Cross_Border', 'Rush_Shipping_High_Discount',
    'Fraud_Risk_Proxy_Score',
    'Customer_Avg_Quantity', 'Customer_Std_Quantity', 'Customer_Total_Quantity',
    'Customer_Avg_Sales', 'Customer_Std_Sales', 'Customer_Total_Sales',
    'Customer_Avg_Discount', 'Customer_Fraud_Rate',
    'Product_Avg_Quantity', 'Product_Total_Quantity', 'Product_Avg_Sales',
    'Product_Total_Sales', 'Product_Fraud_Rate',
    'Region_Avg_Quantity', 'Region_Total_Quantity', 'Region_Avg_Sales',
    'Region_Total_Sales', 'Region_Fraud_Rate',
    'Category_Avg_Quantity', 'Category_Total_Quantity', 'Category_Avg_Sales',
    'Category_Total_Sales', 'Category_Fraud_Rate',
    'Discount_X_Quantity', 'Price_X_Discount', 'Region_X_Payment_Risk'
]

# Encoded categorical features
encoded_features = [f'{col}_Encoded' for col in categorical_cols if f'{col}_Encoded' in df_model.columns]

# All features
all_features = numerical_features + encoded_features

# Handle missing values
for col in all_features:
    if col in df_model.columns:
        df_model[col] = df_model[col].fillna(df_model[col].median())

print(f"✅ Total features: {len(all_features)}")
print(f"   - Numerical: {len(numerical_features)}")
print(f"   - Encoded Categorical: {len(encoded_features)}")

# ──────────────────────────────────────────────────────────────────────────────
# 3.3 OUTLIER HANDLING
# ──────────────────────────────────────────────────────────────────────────────
print("🔍 Handling Outliers...")

def cap_outliers(df, column, lower_quantile=0.01, upper_quantile=0.99):
    lower = df[column].quantile(lower_quantile)
    upper = df[column].quantile(upper_quantile)
    df[column] = df[column].clip(lower, upper)
    return df

outlier_cols = ['Sales', 'Order Item Quantity', 'Price_Per_Unit', 'Monthly_Demand']
for col in outlier_cols:
    if col in df_model.columns:
        df_model = cap_outliers(df_model, col)

print(f"✅ Outliers capped for {len(outlier_cols)} features")

# ──────────────────────────────────────────────────────────────────────────────
# 3.4 PREPARE DATASETS FOR TASKS
# ──────────────────────────────────────────────────────────────────────────────
print("📦 Preparing Datasets for Each Task...")

# Remove any remaining NaN
df_model = df_model.dropna(subset=all_features + ['Monthly_Demand', 'Fraud_Risk_Label'])

# Features matrix
X = df_model[all_features].values

# Task 1: Monthly Demand Forecasting
y_demand = df_model['Monthly_Demand'].values

# Task 2: Fraud Risk Classification
y_fraud = df_model['Fraud_Risk_Label'].values

print(f"✅ Dataset prepared:")
print(f"   - X shape: {X.shape}")
print(f"   - y_demand shape: {y_demand.shape}")
print(f"   - y_fraud shape: {y_fraud.shape}")
print(f"   - Fraud class distribution: {np.bincount(y_fraud)}")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 4: TASK 1 - DEMAND FORECASTING
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📈 PHASE 4: TASK 1 - DEMAND FORECASTING")
print("="*80 + "\n")

# Train-test split
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X, y_demand, test_size=0.2, random_state=42
)

print(f"Train set: {X_train_d.shape}, Test set: {X_test_d.shape}\n")

# Scale features
scaler_demand = RobustScaler()
X_train_d_scaled = scaler_demand.fit_transform(X_train_d)
X_test_d_scaled = scaler_demand.transform(X_test_d)

# Dictionary to store results
demand_results = {}

# ──────────────────────────────────────────────────────────────────────────────
# 4.1 BASELINE: Random Forest
# ──────────────────────────────────────────────────────────────────────────────
print("🌲 Training Baseline: Random Forest Regressor...")

rf_demand = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

rf_demand.fit(X_train_d, y_train_d)
y_pred_rf = rf_demand.predict(X_test_d)

rmse_rf = np.sqrt(mean_squared_error(y_test_d, y_pred_rf))
mae_rf = mean_absolute_error(y_test_d, y_pred_rf)
r2_rf = r2_score(y_test_d, y_pred_rf)
mape_rf = np.mean(np.abs((y_test_d - y_pred_rf) / (y_test_d + 1))) * 100

demand_results['Random Forest'] = {
    'RMSE': rmse_rf,
    'MAE': mae_rf,
    'R2': r2_rf,
    'MAPE': mape_rf
}

print(f"✅ Random Forest Results:")
print(f"   RMSE: {rmse_rf:.4f}")
print(f"   MAE: {mae_rf:.4f}")
print(f"   R²: {r2_rf:.4f}")
print(f"   MAPE: {mape_rf:.4f}%\n")

# ──────────────────────────────────────────────────────────────────────────────
# 4.2 MAIN MODEL: XGBoost with Hyperparameter Tuning
# ──────────────────────────────────────────────────────────────────────────────
print("🚀 Training Main Model: XGBoost with Optuna Tuning...")

def objective_xgb_demand(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
        'tree_method': TREE_METHOD,
        'random_state': 42
    }

    if USE_GPU:
        params['device'] = 'cuda'

    model = XGBRegressor(**params)
    model.fit(X_train_d, y_train_d, eval_set=[(X_test_d, y_test_d)], verbose=False)
    preds = model.predict(X_test_d)
    rmse = np.sqrt(mean_squared_error(y_test_d, preds))
    return rmse

# Run Optuna optimization
study_xgb_demand = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=42)
)
study_xgb_demand.optimize(objective_xgb_demand, n_trials=50, show_progress_bar=True)

print(f"\n🏆 Best XGBoost parameters:")
print(study_xgb_demand.best_params)

# Train final XGBoost model
best_params_xgb_demand = study_xgb_demand.best_params
best_params_xgb_demand['tree_method'] = TREE_METHOD
best_params_xgb_demand['random_state'] = 42
if USE_GPU:
    best_params_xgb_demand['device'] = 'cuda'

xgb_demand = XGBRegressor(**best_params_xgb_demand)
xgb_demand.fit(X_train_d, y_train_d, eval_set=[(X_test_d, y_test_d)], verbose=False)
y_pred_xgb = xgb_demand.predict(X_test_d)

rmse_xgb = np.sqrt(mean_squared_error(y_test_d, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test_d, y_pred_xgb)
r2_xgb = r2_score(y_test_d, y_pred_xgb)
mape_xgb = np.mean(np.abs((y_test_d - y_pred_xgb) / (y_test_d + 1))) * 100

demand_results['XGBoost'] = {
    'RMSE': rmse_xgb,
    'MAE': mae_xgb,
    'R2': r2_xgb,
    'MAPE': mape_xgb
}

print(f"\n✅ XGBoost Results:")
print(f"   RMSE: {rmse_xgb:.4f}")
print(f"   MAE: {mae_xgb:.4f}")
print(f"   R²: {r2_xgb:.4f}")
print(f"   MAPE: {mape_xgb:.4f}%\n")

# ──────────────────────────────────────────────────────────────────────────────
# 4.3 ADVANCED: LightGBM
# ──────────────────────────────────────────────────────────────────────────────
print("💡 Training Advanced Model: LightGBM...")

lgb_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_samples': 20,
    'random_state': 42,
    'verbose': -1
}

if USE_GPU:
    lgb_params['device'] = 'gpu'

lgb_demand = LGBMRegressor(**lgb_params)
lgb_demand.fit(X_train_d, y_train_d)
y_pred_lgb = lgb_demand.predict(X_test_d)

rmse_lgb = np.sqrt(mean_squared_error(y_test_d, y_pred_lgb))
mae_lgb = mean_absolute_error(y_test_d, y_pred_lgb)
r2_lgb = r2_score(y_test_d, y_pred_lgb)
mape_lgb = np.mean(np.abs((y_test_d - y_pred_lgb) / (y_test_d + 1))) * 100

demand_results['LightGBM'] = {
    'RMSE': rmse_lgb,
    'MAE': mae_lgb,
    'R2': r2_lgb,
    'MAPE': mape_lgb
}

print(f"✅ LightGBM Results:")
print(f"   RMSE: {rmse_lgb:.4f}")
print(f"   MAE: {mae_lgb:.4f}")
print(f"   R²: {r2_lgb:.4f}")
print(f"   MAPE: {mape_lgb:.4f}%\n")

# ──────────────────────────────────────────────────────────────────────────────
# 4.4 Summary
# ──────────────────────────────────────────────────────────────────────────────
print("📊 DEMAND FORECASTING SUMMARY:")
print("="*60)
demand_df = pd.DataFrame(demand_results).T
print(demand_df.to_string())
print("="*60 + "\n")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 5: TASK 2 - FRAUD RISK CLASSIFICATION
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔒 PHASE 5: TASK 2 - FRAUD RISK CLASSIFICATION")
print("="*80 + "\n")

# Train-test split
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

print(f"Train set: {X_train_f.shape}, Test set: {X_test_f.shape}")
print(f"Train fraud rate: {y_train_f.mean()*100:.2f}%")
print(f"Test fraud rate: {y_test_f.mean()*100:.2f}%\n")

# Handle class imbalance with SMOTE
print("⚖️ Handling Class Imbalance with SMOTE...")
smote = SMOTE(random_state=42)
X_train_f_balanced, y_train_f_balanced = smote.fit_resample(X_train_f, y_train_f)

print(f"After SMOTE: {X_train_f_balanced.shape}")
print(f"Class distribution: {np.bincount(y_train_f_balanced)}\n")

# Dictionary to store results
fraud_results = {}

# ──────────────────────────────────────────────────────────────────────────────
# 5.1 BASELINE: Logistic Regression
# ──────────────────────────────────────────────────────────────────────────────
print("📊 Training Baseline: Logistic Regression...")

from sklearn.linear_model import LogisticRegression

scaler_fraud_lr = RobustScaler()
X_train_f_scaled = scaler_fraud_lr.fit_transform(X_train_f_balanced)
X_test_f_scaled = scaler_fraud_lr.transform(X_test_f)

lr_fraud = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

lr_fraud.fit(X_train_f_scaled, y_train_f_balanced)
y_pred_lr = lr_fraud.predict(X_test_f_scaled)
y_pred_lr_proba = lr_fraud.predict_proba(X_test_f_scaled)[:, 1]

roc_auc_lr = roc_auc_score(y_test_f, y_pred_lr_proba)
avg_precision_lr = average_precision_score(y_test_f, y_pred_lr_proba)
f1_lr = f1_score(y_test_f, y_pred_lr)
acc_lr = accuracy_score(y_test_f, y_pred_lr)

fraud_results['Logistic Regression'] = {
    'ROC-AUC': roc_auc_lr,
    'Avg Precision': avg_precision_lr,
    'F1-Score': f1_lr,
    'Accuracy': acc_lr
}

print(f"✅ Logistic Regression Results:")
print(f"   ROC-AUC: {roc_auc_lr:.4f}")
print(f"   Avg Precision: {avg_precision_lr:.4f}")
print(f"   F1-Score: {f1_lr:.4f}")
print(f"   Accuracy: {acc_lr:.4f}\n")

# ──────────────────────────────────────────────────────────────────────────────
# 5.2 MAIN MODEL: Random Forest
# ──────────────────────────────────────────────────────────────────────────────
print("🌲 Training Main Model: Random Forest Classifier...")

rf_fraud = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_fraud.fit(X_train_f_balanced, y_train_f_balanced)
y_pred_rf_f = rf_fraud.predict(X_test_f)
y_pred_rf_proba = rf_fraud.predict_proba(X_test_f)[:, 1]

roc_auc_rf = roc_auc_score(y_test_f, y_pred_rf_proba)
avg_precision_rf = average_precision_score(y_test_f, y_pred_rf_proba)
f1_rf_f = f1_score(y_test_f, y_pred_rf_f)
acc_rf_f = accuracy_score(y_test_f, y_pred_rf_f)

fraud_results['Random Forest'] = {
    'ROC-AUC': roc_auc_rf,
    'Avg Precision': avg_precision_rf,
    'F1-Score': f1_rf_f,
    'Accuracy': acc_rf_f
}

print(f"✅ Random Forest Results:")
print(f"   ROC-AUC: {roc_auc_rf:.4f}")
print(f"   Avg Precision: {avg_precision_rf:.4f}")
print(f"   F1-Score: {f1_rf_f:.4f}")
print(f"   Accuracy: {acc_rf_f:.4f}\n")

# ──────────────────────────────────────────────────────────────────────────────
# 5.3 ADVANCED: XGBoost with Tuning
# ──────────────────────────────────────────────────────────────────────────────
print("🚀 Training Advanced Model: XGBoost Classifier...")

def objective_xgb_fraud(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
        'scale_pos_weight': len(y_train_f_balanced[y_train_f_balanced == 0]) / len(y_train_f_balanced[y_train_f_balanced == 1]),
        'tree_method': TREE_METHOD,
        'random_state': 42
    }

    if USE_GPU:
        params['device'] = 'cuda'

    model = XGBClassifier(**params)
    model.fit(X_train_f_balanced, y_train_f_balanced, eval_set=[(X_test_f, y_test_f)], verbose=False)
    preds_proba = model.predict_proba(X_test_f)[:, 1]
    roc_auc = roc_auc_score(y_test_f, preds_proba)
    return roc_auc

study_xgb_fraud = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)
study_xgb_fraud.optimize(objective_xgb_fraud, n_trials=50, show_progress_bar=True)

print(f"\n🏆 Best XGBoost parameters:")
print(study_xgb_fraud.best_params)

# Train final model
best_params_xgb_fraud = study_xgb_fraud.best_params
best_params_xgb_fraud['scale_pos_weight'] = len(y_train_f_balanced[y_train_f_balanced == 0]) / len(y_train_f_balanced[y_train_f_balanced == 1])
best_params_xgb_fraud['tree_method'] = TREE_METHOD
best_params_xgb_fraud['random_state'] = 42
if USE_GPU:
    best_params_xgb_fraud['device'] = 'cuda'

xgb_fraud = XGBClassifier(**best_params_xgb_fraud)
xgb_fraud.fit(X_train_f_balanced, y_train_f_balanced, eval_set=[(X_test_f, y_test_f)], verbose=False)
y_pred_xgb_f = xgb_fraud.predict(X_test_f)
y_pred_xgb_proba = xgb_fraud.predict_proba(X_test_f)[:, 1]

roc_auc_xgb_f = roc_auc_score(y_test_f, y_pred_xgb_proba)
avg_precision_xgb = average_precision_score(y_test_f, y_pred_xgb_proba)
f1_xgb_f = f1_score(y_test_f, y_pred_xgb_f)
acc_xgb_f = accuracy_score(y_test_f, y_pred_xgb_f)

fraud_results['XGBoost'] = {
    'ROC-AUC': roc_auc_xgb_f,
    'Avg Precision': avg_precision_xgb,
    'F1-Score': f1_xgb_f,
    'Accuracy': acc_xgb_f
}

print(f"\n✅ XGBoost Results:")
print(f"   ROC-AUC: {roc_auc_xgb_f:.4f}")
print(f"   Avg Precision: {avg_precision_xgb:.4f}")
print(f"   F1-Score: {f1_xgb_f:.4f}")
print(f"   Accuracy: {acc_xgb_f:.4f}\n")

# ──────────────────────────────────────────────────────────────────────────────
# 5.4 Summary
# ──────────────────────────────────────────────────────────────────────────────
print("📊 FRAUD CLASSIFICATION SUMMARY:")
print("="*60)
fraud_df = pd.DataFrame(fraud_results).T
print(fraud_df.to_string())
print("="*60 + "\n")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 6: MULTI-TASK LEARNING (MTL) - CORE CONTRIBUTION
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔥 PHASE 6: MULTI-TASK LEARNING - THE BREAKTHROUGH")
print("="*80 + "\n")

# ──────────────────────────────────────────────────────────────────────────────
# 6.1 MTL ARCHITECTURE DEFINITION
# ──────────────────────────────────────────────────────────────────────────────

class MultiTaskModel(nn.Module):
    """
    Multi-Task Learning Model with Shared Representations

    Architecture:
    - Shared Feature Extractor (3 layers)
    - Task-Specific Heads:
        * Demand Head (Regression)
        * Fraud Head (Classification)
    """

    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        super(MultiTaskModel, self).__init__()

        # Shared Feature Extractor
        self.shared_layer1 = nn.Linear(input_dim, hidden_dims[0])
        self.bn1 = nn.BatchNorm1d(hidden_dims[0])
        self.dropout1 = nn.Dropout(dropout)

        self.shared_layer2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.bn2 = nn.BatchNorm1d(hidden_dims[1])
        self.dropout2 = nn.Dropout(dropout)

        self.shared_layer3 = nn.Linear(hidden_dims[1], hidden_dims[2])
        self.bn3 = nn.BatchNorm1d(hidden_dims[2])
        self.dropout3 = nn.Dropout(dropout)

        # Task 1: Demand Forecasting Head (Regression)
        self.demand_head = nn.Sequential(
            nn.Linear(hidden_dims[2], 32),
            nn.ReLU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(32, 1)
        )

        # Task 2: Fraud Detection Head (Classification)
        self.fraud_head = nn.Sequential(
            nn.Linear(hidden_dims[2], 32),
            nn.ReLU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        # Shared representation learning
        x = F.relu(self.bn1(self.shared_layer1(x)))
        x = self.dropout1(x)

        x = F.relu(self.bn2(self.shared_layer2(x)))
        x = self.dropout2(x)

        shared_repr = F.relu(self.bn3(self.shared_layer3(x)))
        shared_repr = self.dropout3(shared_repr)

        # Task-specific outputs
        demand_output = self.demand_head(shared_repr)
        fraud_output = self.fraud_head(shared_repr)

        return demand_output, fraud_output, shared_repr

# ──────────────────────────────────────────────────────────────────────────────
# 6.2 PREPARE DATA FOR MTL
# ──────────────────────────────────────────────────────────────────────────────
print("📦 Preparing Data for Multi-Task Learning...")

# Split data
X_train_mtl, X_test_mtl, y_train_d_mtl, y_test_d_mtl, y_train_f_mtl, y_test_f_mtl = train_test_split(
    X, y_demand, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

# Scale features
scaler_mtl = RobustScaler()
X_train_mtl_scaled = scaler_mtl.fit_transform(X_train_mtl)
X_test_mtl_scaled = scaler_mtl.transform(X_test_mtl)

# Normalize demand target
demand_scaler = RobustScaler()
y_train_d_mtl_scaled = demand_scaler.fit_transform(y_train_d_mtl.reshape(-1, 1)).flatten()
y_test_d_mtl_scaled = demand_scaler.transform(y_test_d_mtl.reshape(-1, 1)).flatten()

# Handle class imbalance for fraud task
print("⚖️ Balancing fraud samples for MTL...")
smote_mtl = SMOTE(random_state=42)
X_train_mtl_balanced, y_train_f_mtl_balanced = smote_mtl.fit_resample(X_train_mtl_scaled, y_train_f_mtl)

# Get corresponding demand labels for balanced samples
indices_balanced = smote_mtl.fit_resample(
    np.arange(len(X_train_mtl_scaled)).reshape(-1, 1),
    y_train_f_mtl
)[0].flatten()

y_train_d_mtl_balanced = np.array([y_train_d_mtl_scaled[i % len(y_train_d_mtl_scaled)] for i in indices_balanced])

print(f"✅ MTL Data Ready:")
print(f"   Train: {X_train_mtl_balanced.shape}")
print(f"   Test: {X_test_mtl_scaled.shape}")
print(f"   Fraud balance: {np.bincount(y_train_f_mtl_balanced)}\n")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_mtl_balanced).to(device)
y_train_demand_tensor = torch.FloatTensor(y_train_d_mtl_balanced).to(device)
y_train_fraud_tensor = torch.LongTensor(y_train_f_mtl_balanced).to(device)

X_test_tensor = torch.FloatTensor(X_test_mtl_scaled).to(device)
y_test_demand_tensor = torch.FloatTensor(y_test_d_mtl_scaled).to(device)
y_test_fraud_tensor = torch.LongTensor(y_test_f_mtl).to(device)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_demand_tensor, y_train_fraud_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_demand_tensor, y_test_fraud_tensor)

batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ══════════════════════════════════════════════════════════════════════════════
# 6.3 TRAINING MTL MODEL - FIXED VERSION
# ══════════════════════════════════════════════════════════════════════════════
print("🎯 Training Multi-Task Learning Model...")

# Initialize model
input_dim = X_train_mtl_balanced.shape[1]
mtl_model = MultiTaskModel(
    input_dim=input_dim,
    hidden_dims=[256, 128, 64],
    dropout=0.3
).to(device)

# Loss functions
criterion_demand = nn.MSELoss()
criterion_fraud = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.AdamW(mtl_model.parameters(), lr=0.001, weight_decay=1e-5)

# Learning rate scheduler - FIXED: removed verbose parameter
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

# Training parameters
num_epochs = 100
best_loss = float('inf')
patience = 15
patience_counter = 0

# Task weights
lambda_demand = 1.0
lambda_fraud = 1.0

# Training loop
train_losses = []
test_losses = []

print(f"\n🚀 Starting MTL Training on {device}...\n")

for epoch in range(num_epochs):
    # Training phase
    mtl_model.train()
    train_loss_epoch = 0
    train_demand_loss_epoch = 0
    train_fraud_loss_epoch = 0

    for batch_X, batch_y_demand, batch_y_fraud in train_loader:
        optimizer.zero_grad()

        # Forward pass
        demand_pred, fraud_pred, _ = mtl_model(batch_X)

        # Calculate losses
        loss_demand = criterion_demand(demand_pred.squeeze(), batch_y_demand)
        loss_fraud = criterion_fraud(fraud_pred, batch_y_fraud)

        # Combined loss
        total_loss = lambda_demand * loss_demand + lambda_fraud * loss_fraud

        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(mtl_model.parameters(), max_norm=1.0)
        optimizer.step()

        train_loss_epoch += total_loss.item()
        train_demand_loss_epoch += loss_demand.item()
        train_fraud_loss_epoch += loss_fraud.item()

    # Validation phase
    mtl_model.eval()
    test_loss_epoch = 0
    test_demand_loss_epoch = 0
    test_fraud_loss_epoch = 0

    with torch.no_grad():
        for batch_X, batch_y_demand, batch_y_fraud in test_loader:
            demand_pred, fraud_pred, _ = mtl_model(batch_X)

            loss_demand = criterion_demand(demand_pred.squeeze(), batch_y_demand)
            loss_fraud = criterion_fraud(fraud_pred, batch_y_fraud)
            total_loss = lambda_demand * loss_demand + lambda_fraud * loss_fraud

            test_loss_epoch += total_loss.item()
            test_demand_loss_epoch += loss_demand.item()
            test_fraud_loss_epoch += loss_fraud.item()

    # Calculate average losses
    avg_train_loss = train_loss_epoch / len(train_loader)
    avg_test_loss = test_loss_epoch / len(test_loader)

    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)

    # Learning rate scheduling
    scheduler.step(avg_test_loss)

    # Print progress
    if (epoch + 1) % 10 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch [{epoch+1}/{num_epochs}] - LR: {current_lr:.6f}")
        print(f"  Train Loss: {avg_train_loss:.4f} (Demand: {train_demand_loss_epoch/len(train_loader):.4f}, Fraud: {train_fraud_loss_epoch/len(train_loader):.4f})")
        print(f"  Test Loss:  {avg_test_loss:.4f} (Demand: {test_demand_loss_epoch/len(test_loader):.4f}, Fraud: {test_fraud_loss_epoch/len(test_loader):.4f})")

    # Early stopping
    if avg_test_loss < best_loss:
        best_loss = avg_test_loss
        patience_counter = 0
        # Save best model
        torch.save(mtl_model.state_dict(), 'best_mtl_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\n⚠️ Early stopping triggered at epoch {epoch+1}")
            break

print(f"\n✅ Training completed!")
print(f"   Best test loss: {best_loss:.4f}\n")

# Load best model
mtl_model.load_state_dict(torch.load('best_mtl_model.pth'))

# ──────────────────────────────────────────────────────────────────────────────
# 6.4 EVALUATE MTL MODEL
# ──────────────────────────────────────────────────────────────────────────────
print("📊 Evaluating Multi-Task Learning Model...\n")

mtl_model.eval()
all_demand_preds = []
all_fraud_preds = []
all_fraud_probs = []

with torch.no_grad():
    for batch_X, _, _ in test_loader:
        demand_pred, fraud_pred, _ = mtl_model(batch_X)
        all_demand_preds.append(demand_pred.cpu().numpy())
        all_fraud_preds.append(torch.argmax(fraud_pred, dim=1).cpu().numpy())
        all_fraud_probs.append(F.softmax(fraud_pred, dim=1)[:, 1].cpu().numpy())

# Concatenate predictions
mtl_demand_preds = np.concatenate(all_demand_preds).flatten()
mtl_fraud_preds = np.concatenate(all_fraud_preds)
mtl_fraud_probs = np.concatenate(all_fraud_probs)

# Inverse transform demand predictions
mtl_demand_preds_original = demand_scaler.inverse_transform(mtl_demand_preds.reshape(-1, 1)).flatten()

# Calculate metrics for demand forecasting
mtl_rmse_demand = np.sqrt(mean_squared_error(y_test_d_mtl, mtl_demand_preds_original))
mtl_mae_demand = mean_absolute_error(y_test_d_mtl, mtl_demand_preds_original)
mtl_r2_demand = r2_score(y_test_d_mtl, mtl_demand_preds_original)
mtl_mape_demand = np.mean(np.abs((y_test_d_mtl - mtl_demand_preds_original) / (y_test_d_mtl + 1))) * 100

# Calculate metrics for fraud classification
mtl_roc_auc = roc_auc_score(y_test_f_mtl, mtl_fraud_probs)
mtl_avg_precision = average_precision_score(y_test_f_mtl, mtl_fraud_probs)
mtl_f1 = f1_score(y_test_f_mtl, mtl_fraud_preds)
mtl_accuracy = accuracy_score(y_test_f_mtl, mtl_fraud_preds)

print("🎯 MULTI-TASK LEARNING RESULTS:")
print("="*60)
print("\n📈 TASK 1 - Demand Forecasting:")
print(f"   RMSE: {mtl_rmse_demand:.4f}")
print(f"   MAE: {mtl_mae_demand:.4f}")
print(f"   R²: {mtl_r2_demand:.4f}")
print(f"   MAPE: {mtl_mape_demand:.4f}%")

print("\n🔒 TASK 2 - Fraud Detection:")
print(f"   ROC-AUC: {mtl_roc_auc:.4f}")
print(f"   Avg Precision: {mtl_avg_precision:.4f}")
print(f"   F1-Score: {mtl_f1:.4f}")
print(f"   Accuracy: {mtl_accuracy:.4f}")
print("="*60 + "\n")

# Add MTL results to comparison
demand_results['MTL Model'] = {
    'RMSE': mtl_rmse_demand,
    'MAE': mtl_mae_demand,
    'R2': mtl_r2_demand,
    'MAPE': mtl_mape_demand
}

fraud_results['MTL Model'] = {
    'ROC-AUC': mtl_roc_auc,
    'Avg Precision': mtl_avg_precision,
    'F1-Score': mtl_f1,
    'Accuracy': mtl_accuracy
}

# ──────────────────────────────────────────────────────────────────────────────
# 6.5 COMPARATIVE ANALYSIS: MTL vs Single-Task
# ──────────────────────────────────────────────────────────────────────────────
print("📊 COMPARATIVE ANALYSIS: MTL vs Single-Task Models")
print("="*80)

print("\n🎯 DEMAND FORECASTING COMPARISON:")
demand_comparison = pd.DataFrame(demand_results).T
demand_comparison = demand_comparison.sort_values('RMSE')
print(demand_comparison.to_string())

improvement_demand = ((demand_comparison.loc['XGBoost', 'RMSE'] -
                       demand_comparison.loc['MTL Model', 'RMSE']) /
                      demand_comparison.loc['XGBoost', 'RMSE'] * 100)
print(f"\n💡 MTL Improvement over best single-task (XGBoost): {improvement_demand:.2f}%")

print("\n🔒 FRAUD DETECTION COMPARISON:")
fraud_comparison = pd.DataFrame(fraud_results).T
fraud_comparison = fraud_comparison.sort_values('ROC-AUC', ascending=False)
print(fraud_comparison.to_string())

improvement_fraud = ((fraud_comparison.loc['MTL Model', 'ROC-AUC'] -
                     fraud_comparison.loc['XGBoost', 'ROC-AUC']) /
                    fraud_comparison.loc['XGBoost', 'ROC-AUC'] * 100)
print(f"\n💡 MTL Improvement over best single-task (XGBoost): {improvement_fraud:.2f}%")
print("="*80 + "\n")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 7: TASK 3 - FRAUD ARCHETYPE DISCOVERY (UNSUPERVISED)
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔍 PHASE 7: FRAUD ARCHETYPE DISCOVERY (UNSUPERVISED CLUSTERING)")
print("="*80 + "\n")

# ──────────────────────────────────────────────────────────────────────────────
# 7.1 PREPARE FRAUD-RELATED FEATURES
# ──────────────────────────────────────────────────────────────────────────────
print("📦 Preparing Fraud-Related Features for Clustering...")

# Select high-risk orders (top 30%)
fraud_threshold_clustering = df_model['Fraud_Risk_Proxy_Score'].quantile(0.70)
df_fraud_cluster = df_model[df_model['Fraud_Risk_Proxy_Score'] > fraud_threshold_clustering].copy()

print(f"✅ Selected {len(df_fraud_cluster)} high-risk orders for clustering")

# Key fraud-related features
fraud_features_for_clustering = [
    'Order Item Discount Rate', 'Price_Per_Unit', 'Order Item Quantity',
    'Payment_Risk_Score', 'Shipping_Risk_Score', 'Is_Cross_Border',
    'High_Discount_Flag', 'Extreme_Discount_Flag',
    'Cash_High_Discount', 'Cash_Cross_Border', 'Rush_Shipping_High_Discount',
    'Customer_Fraud_Rate', 'Product_Fraud_Rate', 'Region_Fraud_Rate',
    'Fraud_Risk_Proxy_Score'
]

X_fraud_cluster = df_fraud_cluster[fraud_features_for_clustering].values

# Scale features
scaler_cluster = StandardScaler()
X_fraud_cluster_scaled = scaler_cluster.fit_transform(X_fraud_cluster)

# ──────────────────────────────────────────────────────────────────────────────
# 7.2 DETERMINE OPTIMAL NUMBER OF CLUSTERS
# ──────────────────────────────────────────────────────────────────────────────
print("🔍 Finding Optimal Number of Clusters...")

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

inertias = []
silhouette_scores = []
calinski_scores = []
davies_bouldin_scores = []

K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_fraud_cluster_scaled)

    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_fraud_cluster_scaled, labels))
    calinski_scores.append(calinski_harabasz_score(X_fraud_cluster_scaled, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_fraud_cluster_scaled, labels))

# Find optimal k (highest silhouette score)
optimal_k_idx = np.argmax(silhouette_scores)
optimal_k = list(K_range)[optimal_k_idx]

print(f"\n✅ Optimal number of clusters: {optimal_k}")
print(f"   Silhouette Score: {silhouette_scores[optimal_k_idx]:.4f}")

# ──────────────────────────────────────────────────────────────────────────────
# 7.3 PERFORM CLUSTERING
# ──────────────────────────────────────────────────────────────────────────────
print(f"\n🎯 Performing K-Means Clustering with k={optimal_k}...")

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
cluster_labels = kmeans_final.fit_predict(X_fraud_cluster_scaled)

df_fraud_cluster['Cluster'] = cluster_labels

print(f"✅ Clustering completed!")
print(f"\nCluster distribution:")
print(df_fraud_cluster['Cluster'].value_counts().sort_index())

# ──────────────────────────────────────────────────────────────────────────────
# 7.4 ANALYZE FRAUD ARCHETYPES
# ──────────────────────────────────────────────────────────────────────────────
print("\n📊 FRAUD ARCHETYPE ANALYSIS:")
print("="*80)

archetype_profiles = {}

for cluster_id in range(optimal_k):
    cluster_data = df_fraud_cluster[df_fraud_cluster['Cluster'] == cluster_id]

    print(f"\n🔸 ARCHETYPE {cluster_id + 1} (n={len(cluster_data)})")
    print("-" * 60)

    # Key characteristics
    profile = {
        'Size': len(cluster_data),
        'Avg_Discount': cluster_data['Order Item Discount Rate'].mean(),
        'Avg_Order_Value': cluster_data['Sales'].mean(),
        'Avg_Quantity': cluster_data['Order Item Quantity'].mean(),
        'Cross_Border_Rate': cluster_data['Is_Cross_Border'].mean(),
        'High_Discount_Rate': cluster_data['High_Discount_Flag'].mean(),
        'Cash_Payment_Rate': (cluster_data['Type_Encoded'] == label_encoders['Type'].transform(['CASH'])[0]).mean(),
        'Rush_Shipping_Rate': (cluster_data['Shipping Mode_Encoded'] ==
                               label_encoders['Shipping Mode'].transform(['Same Day'])[0]).mean(),
        'Avg_Risk_Score': cluster_data['Fraud_Risk_Proxy_Score'].mean()
    }

    archetype_profiles[f'Archetype_{cluster_id + 1}'] = profile

    # Print profile
    print(f"  Average Discount Rate: {profile['Avg_Discount']*100:.2f}%")
    print(f"  Average Order Value: ${profile['Avg_Order_Value']:.2f}")
    print(f"  Average Quantity: {profile['Avg_Quantity']:.2f}")
    print(f"  Cross-Border Rate: {profile['Cross_Border_Rate']*100:.2f}%")
    print(f"  High Discount Rate: {profile['High_Discount_Rate']*100:.2f}%")
    print(f"  Cash Payment Rate: {profile['Cash_Payment_Rate']*100:.2f}%")
    print(f"  Rush Shipping Rate: {profile['Rush_Shipping_Rate']*100:.2f}%")
    print(f"  Average Risk Score: {profile['Avg_Risk_Score']:.2f}")

    # Most common characteristics
    print(f"\n  Top Payment Types:")
    print(cluster_data['Type'].value_counts().head(3).to_string())

    print(f"\n  Top Regions:")
    print(cluster_data['Order Region'].value_counts().head(3).to_string())

    print(f"\n  Top Categories:")
    print(cluster_data['Category Name'].value_counts().head(3).to_string())

print("\n" + "="*80)

# ──────────────────────────────────────────────────────────────────────────────
# 7.5 DIMENSIONALITY REDUCTION FOR VISUALIZATION
# ──────────────────────────────────────────────────────────────────────────────
print("\n🎨 Performing Dimensionality Reduction...")

# PCA
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_fraud_cluster_scaled)

print(f"✅ PCA completed")
print(f"   Explained variance: {pca.explained_variance_ratio_.sum()*100:.2f}%")

# t-SNE (sample if too large)
if len(X_fraud_cluster_scaled) > 5000:
    sample_idx = np.random.choice(len(X_fraud_cluster_scaled), 5000, replace=False)
    X_tsne_input = X_fraud_cluster_scaled[sample_idx]
    tsne_labels = cluster_labels[sample_idx]
else:
    X_tsne_input = X_fraud_cluster_scaled
    tsne_labels = cluster_labels

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_tsne_input)

print(f"✅ t-SNE completed")

# Store for visualization later
clustering_results = {
    'pca_coords': X_pca,
    'tsne_coords': X_tsne,
    'cluster_labels': cluster_labels,
    'tsne_labels': tsne_labels,
    'archetype_profiles': archetype_profiles
}

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 8: EXPLAINABILITY WITH SHAP
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔬 PHASE 8: MODEL EXPLAINABILITY WITH SHAP")
print("="*80 + "\n")

# ──────────────────────────────────────────────────────────────────────────────
# 8.1 SHAP FOR DEMAND FORECASTING (XGBoost)
# ──────────────────────────────────────────────────────────────────────────────
print("📊 Computing SHAP Values for Demand Forecasting Model...")

# Sample data for faster computation
sample_size_shap = min(5000, len(X_test_d))
sample_idx_demand = np.random.choice(len(X_test_d), sample_size_shap, replace=False)
X_shap_demand = X_test_d[sample_idx_demand]

# Create SHAP explainer
explainer_demand = shap.TreeExplainer(xgb_demand)
shap_values_demand = explainer_demand.shap_values(X_shap_demand)

print(f"✅ SHAP values computed for demand forecasting")

# Feature importance from SHAP
shap_importance_demand = np.abs(shap_values_demand).mean(axis=0)
feature_importance_demand = pd.DataFrame({
    'Feature': [all_features[i] for i in range(len(all_features))],
    'SHAP_Importance': shap_importance_demand
}).sort_values('SHAP_Importance', ascending=False)

print("\n🔝 Top 15 Features for Demand Forecasting:")
print(feature_importance_demand.head(15).to_string(index=False))

# ──────────────────────────────────────────────────────────────────────────────
# 8.2 SHAP FOR FRAUD DETECTION (XGBoost)
# ──────────────────────────────────────────────────────────────────────────────
print("\n🔒 Computing SHAP Values for Fraud Detection Model...")

sample_idx_fraud = np.random.choice(len(X_test_f), sample_size_shap, replace=False)
X_shap_fraud = X_test_f[sample_idx_fraud]

explainer_fraud = shap.TreeExplainer(xgb_fraud)
shap_values_fraud = explainer_fraud.shap_values(X_shap_fraud)

print(f"✅ SHAP values computed for fraud detection")

# Feature importance from SHAP
if isinstance(shap_values_fraud, list):
    shap_values_fraud_class1 = shap_values_fraud[1]
else:
    shap_values_fraud_class1 = shap_values_fraud

shap_importance_fraud = np.abs(shap_values_fraud_class1).mean(axis=0)
feature_importance_fraud = pd.DataFrame({
    'Feature': [all_features[i] for i in range(len(all_features))],
    'SHAP_Importance': shap_importance_fraud
}).sort_values('SHAP_Importance', ascending=False)

print("\n🔝 Top 15 Features for Fraud Detection:")
print(feature_importance_fraud.head(15).to_string(index=False))

# ──────────────────────────────────────────────────────────────────────────────
# 8.3 KEY INSIGHTS FROM SHAP
# ──────────────────────────────────────────────────────────────────────────────
print("\n" + "="*80)
print("💡 KEY INSIGHTS FROM EXPLAINABILITY ANALYSIS")
print("="*80)

# Find overlapping important features
top_demand_features = set(feature_importance_demand.head(20)['Feature'])
top_fraud_features = set(feature_importance_fraud.head(20)['Feature'])
shared_important_features = top_demand_features.intersection(top_fraud_features)

print(f"\n🔗 Shared Important Features (appear in top 20 for both tasks):")
for feat in shared_important_features:
    demand_rank = feature_importance_demand[feature_importance_demand['Feature'] == feat].index[0] + 1
    fraud_rank = feature_importance_fraud[feature_importance_fraud['Feature'] == feat].index[0] + 1
    print(f"  • {feat}")
    print(f"    - Demand rank: #{demand_rank}, Fraud rank: #{fraud_rank}")

print(f"\n📈 Business Implications:")
print("  • Discount rate is critical for BOTH demand forecasting and fraud detection")
print("  • Regional patterns influence both legitimate demand and fraud risk")
print("  • Payment types serve as strong fraud indicators but less impact on demand")
print("  • Customer behavior aggregations help identify both trends and anomalies")

# Store SHAP results
shap_results = {
    'demand_shap_values': shap_values_demand,
    'fraud_shap_values': shap_values_fraud_class1,
    'X_shap_demand': X_shap_demand,
    'X_shap_fraud': X_shap_fraud,
    'feature_importance_demand': feature_importance_demand,
    'feature_importance_fraud': feature_importance_fraud,
    'shared_features': list(shared_important_features)
}

print("="*80 + "\n")

# ══════════════════════════════════════════════════════════════════════════════
# PHASE 9: SAVE RESULTS & MODELS
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("💾 PHASE 9: SAVING RESULTS & MODELS")
print("="*80 + "\n")

# Create results directory
import os
os.makedirs('results', exist_ok=True)

# ──────────────────────────────────────────────────────────────────────────────
# 9.1 SAVE PERFORMANCE METRICS
# ──────────────────────────────────────────────────────────────────────────────
print("📊 Saving Performance Metrics...")

# Demand forecasting results
demand_comparison.to_csv('results/demand_forecasting_results.csv')

# Fraud detection results
fraud_comparison.to_csv('results/fraud_detection_results.csv')

# Archetype profiles
pd.DataFrame(archetype_profiles).T.to_csv('results/fraud_archetype_profiles.csv')

# Feature importance
feature_importance_demand.to_csv('results/feature_importance_demand.csv', index=False)
feature_importance_fraud.to_csv('results/feature_importance_fraud.csv', index=False)

print("✅ Performance metrics saved")

# ──────────────────────────────────────────────────────────────────────────────
# 9.2 SAVE MODELS
# ──────────────────────────────────────────────────────────────────────────────
print("🤖 Saving Trained Models...")

# Save tree-based models
pickle.dump(xgb_demand, open('results/xgb_demand_model.pkl', 'wb'))
pickle.dump(xgb_fraud, open('results/xgb_fraud_model.pkl', 'wb'))
pickle.dump(lgb_demand, open('results/lgb_demand_model.pkl', 'wb'))
pickle.dump(rf_demand, open('results/rf_demand_model.pkl', 'wb'))
pickle.dump(rf_fraud, open('results/rf_fraud_model.pkl', 'wb'))

# Save scalers
pickle.dump(scaler_demand, open('results/scaler_demand.pkl', 'wb'))
pickle.dump(scaler_mtl, open('results/scaler_mtl.pkl', 'wb'))
pickle.dump(demand_scaler, open('results/demand_target_scaler.pkl', 'wb'))

# Save label encoders
pickle.dump(label_encoders, open('results/label_encoders.pkl', 'wb'))

print("✅ Models saved")

# ──────────────────────────────────────────────────────────────────────────────
# 9.3 SAVE CLUSTERING RESULTS
# ──────────────────────────────────────────────────────────────────────────────
print("🔍 Saving Clustering Results...")

df_fraud_cluster[['Cluster'] + fraud_features_for_clustering].to_csv(
    'results/fraud_clusters.csv', index=False
)

pickle.dump(clustering_results, open('results/clustering_results.pkl', 'wb'))
pickle.dump(kmeans_final, open('results/kmeans_model.pkl', 'wb'))

print("✅ Clustering results saved")

# ──────────────────────────────────────────────────────────────────────────────
# 9.4 SAVE SHAP RESULTS
# ──────────────────────────────────────────────────────────────────────────────
print("🔬 Saving SHAP Results...")

pickle.dump(shap_results, open('results/shap_results.pkl', 'wb'))

print("✅ SHAP results saved")

# ──────────────────────────────────────────────────────────────────────────────
# 9.5 CREATE SUMMARY REPORT
# ──────────────────────────────────────────────────────────────────────────────
print("📝 Creating Summary Report...")

summary_report = {
    'Project': 'Explainable Multi-Task Learning for Supply Chain',
    'Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Dataset_Shape': df.shape,
    'Total_Features': len(all_features),
    'Demand_Forecasting': {
        'Best_Model': 'MTL Model' if mtl_rmse_demand < rmse_xgb else 'XGBoost',
        'Best_RMSE': min(mtl_rmse_demand, rmse_xgb),
        'Best_R2': max(mtl_r2_demand, r2_xgb),
        'MTL_vs_XGBoost_Improvement': improvement_demand
    },
    'Fraud_Detection': {
        'Best_Model': 'MTL Model' if mtl_roc_auc > roc_auc_xgb_f else 'XGBoost',
        'Best_ROC_AUC': max(mtl_roc_auc, roc_auc_xgb_f),
        'Best_F1': max(mtl_f1, f1_xgb_f),
        'MTL_vs_XGBoost_Improvement': improvement_fraud
    },
    'Fraud_Archetypes': {
        'Num_Clusters': optimal_k,
        'Silhouette_Score': silhouette_scores[optimal_k_idx],
        'High_Risk_Orders_Analyzed': len(df_fraud_cluster)
    },
    'Top_Shared_Features': list(shared_important_features)[:10]
}

with open('results/summary_report.json', 'w') as f:
    json.dump(summary_report, f, indent=4)

print("✅ Summary report created")

print("\n" + "="*80)
print("✅ ALL RESULTS SAVED TO 'results/' DIRECTORY")
print("="*80 + "\n")

print("\n🏆 PROJECT COMPLETE!")
print("\n📁 OUTPUT FILES:")
print("  • results/demand_forecasting_results.csv")
print("  • results/fraud_detection_results.csv")
print("  • results/fraud_archetype_profiles.csv")
print("  • results/feature_importance_demand.csv")
print("  • results/feature_importance_fraud.csv")
print("  • results/summary_report.json")
print("  • results/*.pkl (models and data)")
print("  • best_mtl_model.pth (PyTorch MTL model)")

print("\n✨ Chúc bạn thành công với đồ án! ✨\n")

🔥 Using device: cuda
GPU: Tesla T4
Memory: 15.83 GB

📊 PHASE 1: DATA LOADING & EXPLORATION

✅ Dataset loaded successfully!
📈 Shape: (2000, 22)
📝 Columns: ['Type', 'Days for shipment (scheduled)', 'Category Name', 'Customer City', 'Customer Country', 'Customer Segment', 'Customer State', 'Department Name', 'Market', 'Order City', 'Order Country', 'Order Item Discount Rate', 'Order Item Quantity', 'Sales', 'Order Region', 'Order State', 'Product Name', 'Shipping Mode', 'Day of Week', 'Month', 'Year', 'Week of Year']

🔍 Data Types:
Type                              object
Days for shipment (scheduled)      int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Segment                  object
Customer State                    object
Department Name                   object
Market                            object
Order City                        object
Order Country                     object
Order Item Dis

[I 2026-01-30 13:38:14,934] A new study created in memory with name: no-name-730191af-c71f-435f-ad4f-90bba49a828c


✅ Random Forest Results:
   RMSE: 4.6847
   MAE: 2.8165
   R²: 0.9313
   MAPE: 36.0073%

🚀 Training Main Model: XGBoost with Optuna Tuning...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-01-30 13:38:15,844] Trial 0 finished with value: 4.4397989684636725 and parameters: {'n_estimators': 250, 'max_depth': 12, 'learning_rate': 0.22227824312530747, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.02904180608409973, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 4.4397989684636725.
[I 2026-01-30 13:38:16,773] Trial 1 finished with value: 4.982779029118926 and parameters: {'n_estimators': 383, 'max_depth': 3, 'learning_rate': 0.29127385712697834, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'min_child_weight': 2, 'gamma': 0.09170225492671691, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 0 with value: 4.4397989684636725.
[I 2026-01-30 13:38:17,478] Trial 2 finished with value: 4.518241573027216 and parameters: {'n_estimators': 273, 'max_depth': 5, 'learning_rate': 0.18743733946949004, 'subsample': 

[I 2026-01-30 13:38:55,447] A new study created in memory with name: no-name-6b23165e-0a33-4c84-83ac-cf465d619f57


✅ Random Forest Results:
   ROC-AUC: 1.0000
   Avg Precision: 1.0000
   F1-Score: 1.0000
   Accuracy: 1.0000

🚀 Training Advanced Model: XGBoost Classifier...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-01-30 13:38:55,796] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 250, 'max_depth': 12, 'learning_rate': 0.22227824312530747, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.02904180608409973, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 1.0.
[I 2026-01-30 13:38:56,239] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 383, 'max_depth': 3, 'learning_rate': 0.29127385712697834, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'min_child_weight': 2, 'gamma': 0.09170225492671691, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 0 with value: 1.0.
[I 2026-01-30 13:38:56,638] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 273, 'max_depth': 5, 'learning_rate': 0.18743733946949004, 'subsample': 0.6557975442608167, 'colsample_bytree': 0.7168578594140873, 'min_child_we

In [4]:
# ══════════════════════════════════════════════════════════════════════════════
# VISUALIZATION: 2 KEY FIGURES FOR LATEX REPORT
# ══════════════════════════════════════════════════════════════════════════════

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 8)

# ══════════════════════════════════════════════════════════════════════════════
# FIGURE 1: MULTI-TASK LEARNING ARCHITECTURE & PERFORMANCE COMPARISON
# ══════════════════════════════════════════════════════════════════════════════

fig1 = plt.figure(figsize=(18, 8))

# ──────────────────────────────────────────────────────────────────────────────
# LEFT: MTL Architecture Diagram
# ──────────────────────────────────────────────────────────────────────────────
ax1 = plt.subplot(1, 3, 1)
ax1.axis('off')

# Architecture components
layer_y = [0.9, 0.7, 0.5, 0.3, 0.15, 0.0]
layer_labels = ['Input Features\n(69 features)',
                'Shared Layer 1\n(256 units)',
                'Shared Layer 2\n(128 units)',
                'Shared Layer 3\n(64 units)',
                'Task-Specific Heads',
                'Outputs']

colors_arch = ['#E8F4F8', '#B3E5FC', '#81D4FA', '#4FC3F7', '#FFE082', '#C8E6C9']

# Draw architecture
for i, (y, label, color) in enumerate(zip(layer_y, layer_labels, colors_arch)):
    if i < 4:  # Shared layers
        rect = plt.Rectangle((0.1, y-0.05), 0.8, 0.08,
                             facecolor=color, edgecolor='black', linewidth=2)
        ax1.add_patch(rect)
        ax1.text(0.5, y, label, ha='center', va='center',
                fontsize=11, fontweight='bold')
    elif i == 4:  # Task heads
        # Demand head
        rect1 = plt.Rectangle((0.05, y-0.05), 0.35, 0.08,
                              facecolor=colors_arch[4], edgecolor='black', linewidth=2)
        ax1.add_patch(rect1)
        ax1.text(0.225, y, 'Demand Head\n(Regression)', ha='center', va='center',
                fontsize=10, fontweight='bold')

        # Fraud head
        rect2 = plt.Rectangle((0.6, y-0.05), 0.35, 0.08,
                              facecolor=colors_arch[5], edgecolor='black', linewidth=2)
        ax1.add_patch(rect2)
        ax1.text(0.775, y, 'Fraud Head\n(Classification)', ha='center', va='center',
                fontsize=10, fontweight='bold')
    else:  # Outputs
        # Demand output
        ax1.text(0.225, y, 'Monthly\nDemand', ha='center', va='center',
                fontsize=10, style='italic', color='#F57C00')
        # Fraud output
        ax1.text(0.775, y, 'Fraud Risk\nScore', ha='center', va='center',
                fontsize=10, style='italic', color='#388E3C')

# Arrows
arrow_props = dict(arrowstyle='->', lw=2, color='gray')
for i in range(len(layer_y)-2):
    if i < 3:  # Shared layers arrows
        ax1.annotate('', xy=(0.5, layer_y[i+1]+0.03), xytext=(0.5, layer_y[i]-0.03),
                    arrowprops=arrow_props)
    elif i == 3:  # Split to task heads
        ax1.annotate('', xy=(0.225, layer_y[i+1]+0.03), xytext=(0.4, layer_y[i]-0.03),
                    arrowprops=arrow_props)
        ax1.annotate('', xy=(0.775, layer_y[i+1]+0.03), xytext=(0.6, layer_y[i]-0.03),
                    arrowprops=arrow_props)
    else:  # To outputs
        ax1.annotate('', xy=(0.225, layer_y[i+1]+0.02), xytext=(0.225, layer_y[i]-0.03),
                    arrowprops=arrow_props)
        ax1.annotate('', xy=(0.775, layer_y[i+1]+0.02), xytext=(0.775, layer_y[i]-0.03),
                    arrowprops=arrow_props)

ax1.set_xlim(0, 1)
ax1.set_ylim(-0.05, 1)
ax1.set_title('(A) Multi-Task Learning Architecture', fontsize=14, fontweight='bold', pad=20)

# ──────────────────────────────────────────────────────────────────────────────
# MIDDLE: Demand Forecasting Performance Comparison
# ──────────────────────────────────────────────────────────────────────────────
ax2 = plt.subplot(1, 3, 2)

# Load results
demand_results = pd.read_csv('results/demand_forecasting_results.csv', index_col=0)

models = demand_results.index.tolist()
rmse_values = demand_results['RMSE'].values
r2_values = demand_results['R2'].values

x = np.arange(len(models))
width = 0.35

# Create bars
bars1 = ax2.bar(x - width/2, rmse_values, width, label='RMSE ↓',
                color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A'])
bars2_ax = ax2.twinx()
bars2 = bars2_ax.bar(x + width/2, r2_values, width, label='R² ↑',
                     color=['#95E1D3', '#F38181', '#AA96DA', '#FCBAD3'])

# Customize
ax2.set_xlabel('Models', fontsize=12, fontweight='bold')
ax2.set_ylabel('RMSE (Lower is Better)', fontsize=11, fontweight='bold', color='#FF6B6B')
bars2_ax.set_ylabel('R² Score (Higher is Better)', fontsize=11, fontweight='bold', color='#95E1D3')
ax2.set_title('(B) Demand Forecasting Performance', fontsize=14, fontweight='bold', pad=20)
ax2.set_xticks(x)
ax2.set_xticklabels(models, rotation=15, ha='right')
ax2.tick_params(axis='y', labelcolor='#FF6B6B')
bars2_ax.tick_params(axis='y', labelcolor='#95E1D3')
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

for bar in bars2:
    height = bar.get_height()
    bars2_ax.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Legends
ax2.legend(loc='upper left', fontsize=10)
bars2_ax.legend(loc='upper right', fontsize=10)

# Highlight best model
best_idx = rmse_values.argmin()
ax2.axvspan(best_idx - 0.4, best_idx + 0.4, alpha=0.2, color='gold')

# ──────────────────────────────────────────────────────────────────────────────
# RIGHT: Fraud Detection Performance Comparison
# ──────────────────────────────────────────────────────────────────────────────
ax3 = plt.subplot(1, 3, 3)

# Load results
fraud_results = pd.read_csv('results/fraud_detection_results.csv', index_col=0)

models_fraud = fraud_results.index.tolist()
roc_auc = fraud_results['ROC-AUC'].values
f1_score = fraud_results['F1-Score'].values

x_fraud = np.arange(len(models_fraud))

# Create grouped bars
bars3 = ax3.bar(x_fraud - width/2, roc_auc, width, label='ROC-AUC',
                color=['#667BC6', '#DA7297', '#FADA7A', '#82CD47'])
bars4 = ax3.bar(x_fraud + width/2, f1_score, width, label='F1-Score',
                color=['#C1ADEB', '#FFC7ED', '#FFE5B4', '#BFFCC6'])

# Customize
ax3.set_xlabel('Models', fontsize=12, fontweight='bold')
ax3.set_ylabel('Score (Higher is Better)', fontsize=11, fontweight='bold')
ax3.set_title('(C) Fraud Detection Performance', fontsize=14, fontweight='bold', pad=20)
ax3.set_xticks(x_fraud)
ax3.set_xticklabels(models_fraud, rotation=15, ha='right')
ax3.set_ylim([0.98, 1.002])
ax3.grid(axis='y', alpha=0.3)
ax3.legend(loc='lower right', fontsize=10)

# Add value labels
for bars in [bars3, bars4]:
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}', ha='center', va='bottom', fontsize=8, fontweight='bold')

# Add perfect score line
ax3.axhline(y=1.0, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Perfect Score')

plt.tight_layout()
plt.savefig('results/figure1_mtl_architecture_performance.png',
            dpi=300, bbox_inches='tight', facecolor='white')
print("✅ Figure 1 saved: results/figure1_mtl_architecture_performance.png")
plt.close()

# ══════════════════════════════════════════════════════════════════════════════
# FIGURE 2: FRAUD ARCHETYPE DISCOVERY & SHAP EXPLAINABILITY
# ══════════════════════════════════════════════════════════════════════════════

fig2 = plt.figure(figsize=(18, 8))

# Load data
clustering_results = pickle.load(open('results/clustering_results.pkl', 'rb'))
shap_results = pickle.load(open('results/shap_results.pkl', 'rb'))

# ──────────────────────────────────────────────────────────────────────────────
# LEFT: Fraud Archetypes (PCA Visualization)
# ──────────────────────────────────────────────────────────────────────────────
ax4 = plt.subplot(1, 3, 1)

pca_coords = clustering_results['pca_coords']
cluster_labels = clustering_results['cluster_labels']
archetype_profiles = clustering_results['archetype_profiles']

# Plot clusters
scatter = ax4.scatter(pca_coords[:, 0], pca_coords[:, 1],
                     c=cluster_labels, cmap='Set1',
                     s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

# Add cluster centers
for i in range(len(archetype_profiles)):
    cluster_points = pca_coords[cluster_labels == i]
    center = cluster_points.mean(axis=0)
    ax4.scatter(center[0], center[1], c='black', s=300,
               marker='*', edgecolors='yellow', linewidth=2, zorder=10)
    ax4.text(center[0], center[1] - 0.5, f'Archetype {i+1}\n(n={len(cluster_points)})',
            ha='center', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax4.set_xlabel('Principal Component 1', fontsize=11, fontweight='bold')
ax4.set_ylabel('Principal Component 2', fontsize=11, fontweight='bold')
ax4.set_title('(A) Fraud Archetype Clustering (PCA)', fontsize=14, fontweight='bold', pad=20)
ax4.grid(True, alpha=0.3)
ax4.legend(*scatter.legend_elements(), title="Archetype", loc='best', fontsize=9)

# ──────────────────────────────────────────────────────────────────────────────
# MIDDLE: Archetype Profiles Heatmap
# ──────────────────────────────────────────────────────────────────────────────
ax5 = plt.subplot(1, 3, 2)

# Create profile comparison
archetype_df = pd.DataFrame(archetype_profiles).T
profile_features = ['Avg_Discount', 'Cross_Border_Rate', 'High_Discount_Rate',
                   'Cash_Payment_Rate', 'Rush_Shipping_Rate', 'Avg_Risk_Score']
archetype_subset = archetype_df[profile_features]

# Normalize for better visualization
from sklearn.preprocessing import MinMaxScaler
scaler_viz = MinMaxScaler()
archetype_normalized = pd.DataFrame(
    scaler_viz.fit_transform(archetype_subset.T).T,
    columns=archetype_subset.columns,
    index=archetype_subset.index
)

# Plot heatmap
sns.heatmap(archetype_normalized.T, annot=True, fmt='.2f', cmap='RdYlGn_r',
            cbar_kws={'label': 'Normalized Score'}, linewidths=2, linecolor='white',
            ax=ax5, vmin=0, vmax=1)

ax5.set_xlabel('Archetype', fontsize=11, fontweight='bold')
ax5.set_ylabel('Fraud Characteristics', fontsize=11, fontweight='bold')
ax5.set_title('(B) Archetype Profile Comparison', fontsize=14, fontweight='bold', pad=20)
ax5.set_xticklabels(['Archetype 1\n(Transfer/Debit)', 'Archetype 2\n(Cash)'], rotation=0)
ax5.set_yticklabels(['Avg Discount', 'Cross-Border', 'High Discount',
                     'Cash Payment', 'Rush Shipping', 'Risk Score'], rotation=0)

# ──────────────────────────────────────────────────────────────────────────────
# RIGHT: SHAP Feature Importance for Fraud Detection
# ──────────────────────────────────────────────────────────────────────────────
ax6 = plt.subplot(1, 3, 3)

# Get top 10 features
feature_importance_fraud = shap_results['feature_importance_fraud']
top_features = feature_importance_fraud.head(10).copy()

# Clean feature names
feature_name_map = {
    'Fraud_Risk_Proxy_Score': 'Fraud Risk\nProxy',
    'Order Item Discount Rate': 'Discount\nRate',
    'High_Discount_Flag': 'High Discount\nFlag',
    'Region_X_Payment_Risk': 'Region×Payment\nRisk',
    'Payment_Risk_Score': 'Payment\nRisk',
    'Discount_X_Quantity': 'Discount×\nQuantity',
    'Days for shipment (scheduled)': 'Shipping\nDays',
    'Customer_Fraud_Rate': 'Customer\nFraud Rate',
    'Type_Encoded': 'Payment\nType',
    'Shipping_Risk_Score': 'Shipping\nRisk'
}

top_features['Feature_Clean'] = top_features['Feature'].map(
    lambda x: feature_name_map.get(x, x[:15])
)

# Create horizontal bar chart
colors_shap = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))
bars = ax6.barh(range(len(top_features)), top_features['SHAP_Importance'],
                color=colors_shap, edgecolor='black', linewidth=1.5)

ax6.set_yticks(range(len(top_features)))
ax6.set_yticklabels(top_features['Feature_Clean'][::-1], fontsize=10)
ax6.set_xlabel('Mean |SHAP Value|', fontsize=11, fontweight='bold')
ax6.set_title('(C) Top 10 Fraud Detection Features\n(SHAP Importance)',
             fontsize=14, fontweight='bold', pad=20)
ax6.grid(axis='x', alpha=0.3)
ax6.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_features['SHAP_Importance'])):
    ax6.text(val, bar.get_y() + bar.get_height()/2, f'{val:.2f}',
            va='center', ha='left', fontsize=9, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.savefig('results/figure2_fraud_archetypes_shap.png',
            dpi=300, bbox_inches='tight', facecolor='white')
print("✅ Figure 2 saved: results/figure2_fraud_archetypes_shap.png")
plt.close()

print("\n" + "="*80)
print("🎨 VISUALIZATION COMPLETE!")
print("="*80)
print("\n📊 Generated 2 Key Figures:")
print("  1. figure1_mtl_architecture_performance.png")
print("     - MTL architecture diagram")
print("     - Demand forecasting comparison")
print("     - Fraud detection comparison")
print("\n  2. figure2_fraud_archetypes_shap.png")
print("     - Fraud archetype clustering (PCA)")
print("     - Archetype profile heatmap")
print("     - SHAP feature importance")
print("\n✨ Ready for LaTeX report insertion!")

✅ Figure 1 saved: results/figure1_mtl_architecture_performance.png
✅ Figure 2 saved: results/figure2_fraud_archetypes_shap.png

🎨 VISUALIZATION COMPLETE!

📊 Generated 2 Key Figures:
  1. figure1_mtl_architecture_performance.png
     - MTL architecture diagram
     - Demand forecasting comparison
     - Fraud detection comparison

  2. figure2_fraud_archetypes_shap.png
     - Fraud archetype clustering (PCA)
     - Archetype profile heatmap
     - SHAP feature importance

✨ Ready for LaTeX report insertion!
